<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Relatedness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%run '/content/drive/MyDrive/ColabNotebooks/Main.ipynb'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.8 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=7ff27afd50bf109422c0ac3ecf3c8a2b35b3cb34a44531e9ae32b8469da06e08
  Stored in dir

AttributeError: ignored

AttributeError: ignored

In [4]:
from transformers import LongformerTokenizer, LongformerModel

# Initialize the Longformer tokenizer and model
model_name = "allenai/longformer-base-4096"
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerModel.from_pretrained(model_name)

In [5]:
# List to store encoded data
encoded_data = []

# Your dataset of text, e.g., comments, replies, submissions
text_data = []
for comment in flat_comments_list:
    text_data.append(comment['body'])

for comment in flat_comments_list:
    # Tokenize and encode the text
    encoded = tokenizer.encode(comment['body'], add_special_tokens=True, max_length=4096, truncation=True, return_tensors="pt")

    # Append the encoded text to the list
    # encoded_data.append(input_ids)
    comment['body_encoded'] = encoded

In [6]:
flat_comments_list[:3]

[{'id': 'xsyg0d',
  'body': '',
  'author': None,
  'reply_to': None,
  'conversation_id': 'xsyg0d',
  'body_encoded': tensor([[0, 2]])},
 {'id': 'iqmwd8f',
  'body': "I've been out looking for Ghoulia today, good to know why I couldn't find her.",
  'author': 't2_57iup',
  'reply_to': 'xsyg0d',
  'conversation_id': 'xsyg0d',
  'body_encoded': tensor([[   0,  100,  348,   57,   66,  546,   13, 5977, 5156,  493,  452,    6,
            205,    7,  216,  596,   38, 1705,   75,  465,   69,    4,    2]])},
 {'id': 'iqn4okm',
  'body': 'yeah, fl really got screwed over. was hoping to find my ghouls today but not even a peep, and the workers didn’t even know when they’d be getting stock. i just hope they go online soon',
  'author': 't2_mwcj2qdc',
  'reply_to': 'xsyg0d',
  'conversation_id': 'xsyg0d',
  'body_encoded': tensor([[    0, 42803,     6,  2342,   269,   300, 34797,    81,     4,    21,
            2818,     7,   465,   127, 34648, 44591,   452,    53,    45,   190,
              1

In [7]:
# Find and count long comments
long_comments = [comment for comment in text_data if len(comment) > 4096]

# Print the number of long comments
print(f"Number of long comments: {len(long_comments)}")

# Print the total number of comments
print(f"Total number of comments: {len(text_data)}")

# Filter out comments made by bots
filtered_comments = [comment for comment in text_data if "I am a bot, and this action was performed automatically" not in comment]
non_bot_long_comments = [comment for comment in filtered_comments if len(comment) > 4096]

# Print the number of non-bot comments
print(f"Number of non-bot comments: {len(filtered_comments)}")
print(f"Number of non-bot long comments: {len(non_bot_long_comments)}")

# Print the content of the first 10 long comments
for idx, comment in enumerate(non_bot_long_comments[:10]):
    print(f"Long Comment {idx + 1}:")
    print(comment)
    print("\n")


Number of long comments: 12
Total number of comments: 36685
Number of non-bot comments: 36685
Number of non-bot long comments: 12
Long Comment 1:
i took my sweet time (over a week, w00t) because i was analyzing the precious feedback i got from those brave souls who dared to power through this block of text of mine -- and i was analyzing *hard*. i think i finally got it, so instead of responding to each one of my delicious feedback providers one by one, i'll post my for-now-final conclusions here.

i think i'll solve the dissonance problem i was complaining about by **splitting this bloated thought construct in two** (but still keeping it somewhat connected).

first, the game about space, isolation, horror, distorted signals, and whatnot. i'll keep the design principles as i originally envisioned them: PCs are vat-born beings waking up deep in the bowels of a dead universe. who are they? who sent them? well, they can both hypnonaut and physically break into the space junk around them, s

In [10]:
import torch

# inputs_embeddings = []
counter = 0
for comment in flat_comments_list:
  # Convert input_ids to a PyTorch tensor
  # Modify this line to create a tensor using .clone().detach()
  data = comment['body_encoded'].clone().detach()
  # Forward pass through BERT
  with torch.no_grad():
      outputs = model(data) #.unsqueeze(0))  Add batch dimension (1) for single input

  # Extract embeddings for the [CLS] token (the first token in the sequence)
  comment['body_embeddings'] = outputs[0][:, 0, :].numpy()
  counter += 1
  print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: ignored